In [3]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

/Users/shelbyjouppi/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/shelbyjouppi/.pyenv/versions/3.9.7/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
!pip install fuzzywuzzy


You should consider upgrading via the '/Users/shelbyjouppi/.pyenv/versions/3.9.7/bin/python3.9 -m pip install --upgrade pip' command.


In [5]:
nycClinics = pd.read_csv('nyc_child_adolescent_clinics_geocoded.csv')
fidelis = pd.read_csv('fideliscare-network-scrape.csv')
metroplus = pd.read_csv('metroplus-network-scrape-clean.csv')
healthfirst = pd.read_csv('healthfirst-network-clean.csv')

In [6]:
metroplus = metroplus.drop({'row'},axis=1)

In [7]:
nycClinics

,program_region,program_county,sponsor_name,agency_name,site_name,main_site,site_address,site_city,site_state,site_zip_code,site_county,population_simple,type,full_address,lat,long,geometry
0,New York City,Bronx,"Acacia Network, Inc.","PROMESA, Inc.",Westchester Center of Excellence,Y,915 Westchester Avenue 2nd Floor,Bronx,NY,10459,Bronx,Children & Adolescents,Community,"915 Westchester Avenue 2nd Floor, Bronx, NY 10459",40.821566,-73.898408,"[-73.8984076, 40.8215662]"
1,New York City,Bronx,Association to Benefit Children,Association to Benefit Children,Fast Break - The Bronx,Y,1366 Inwood Avenue,Bronx,NY,10452,Bronx,Children & Adolescents,Community,"1366 Inwood Avenue, Bronx, NY 10452",40.839864,-73.918820,"[-73.9188205, 40.8398641]"
2,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor @ Mott Hall V,N,1551 East 172nd Street Room 216C,Bronx,NY,10472,Bronx,Children & Adolescents,Community,"1551 East 172nd Street Room 216C, Bronx, NY 10472",40.831887,-73.877282,"[-73.8772819, 40.8318874]"
3,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor Clinic Treatment Program - Tilden,Y,750 Tilden Street,Bronx,NY,10467,Bronx,Children & Adolescents,Community,"750 Tilden Street, Bronx, NY 10467",40.876675,-73.862719,"[-73.862719, 40.8766745]"
4,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,"Astor Services @ MS 80, The Isabel Rooney School",N,149 East Mosholu Parkway Room 414C,Bronx,NY,10467,Bronx,Adolescents,School,"149 East Mosholu Parkway Room 414C, Bronx, NY ...",40.876802,-73.881370,"[-73.8813704, 40.8768024]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Port Richmond Satellite,N,166 Lockman Avenue Room 16,Staten Island,NY,10303,Richmond,Children,Community,"166 Lockman Avenue Room 16, Staten Island, NY ...",40.633147,-74.162037,"[-74.16203709999999, 40.6331468]"
483,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Pouch Satellite,N,657 Castleton Avenue Administrative Building,Staten Island,NY,10301,Richmond,Children,Community,"657 Castleton Avenue Administrative Building, ...",40.635312,-74.103344,"[-74.1033443, 40.6353117]"
484,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,SIMHS Chait Clinic of RUMC @ Pediat,N,800 Castleton Avenue 2nd Floor,Staten Island,NY,10310,Richmond,Children & Adolescents,School,"800 Castleton Avenue 2nd Floor, Staten Island,...",40.634491,-74.109268,"[-74.1092682, 40.63449079999999]"
485,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Staten Island MH Society CHAIT Clinic of RUMC,Y,669 Castleton Avenue,Staten Island,NY,10301,Richmond,Children & Adolescents,School,"669 Castleton Avenue, Staten Island, NY 10301",40.635264,-74.103762,"[-74.10376219999999, 40.6352635]"


In [8]:
nycClinics.site_address = nycClinics.site_address.str.lower()
nycClinics.site_address = nycClinics.site_address.str.replace("avenue", "ave")
nycClinics.site_address = nycClinics.site_address.str.replace("street", "st")
nycClinics.site_address = nycClinics.site_address.str.replace(".", "")
nycClinics.site_address = nycClinics.site_address.str.replace("-", "")
nycClinics.site_address = nycClinics.site_address.str.replace(" east ", " e ")
nycClinics.site_address = nycClinics.site_address.str.replace(" north ", " n ")
nycClinics.site_address = nycClinics.site_address.str.replace(" west ", " w ")
nycClinics.site_address = nycClinics.site_address.str.replace(" south ", " s ")
nycClinics.site_address = nycClinics.site_address.str.replace(" road", " rd")


nycClinics.site_address

/var/folders/_4/_njb58q944xdbv6xcbzbzmqh0000gn/T/ipykernel_56730/4224095152.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nycClinics.site_address = nycClinics.site_address.str.replace(".", "")


0                  915 westchester ave 2nd floor
1                                1366 inwood ave
2                      1551 e 172nd st room 216c
3                                  750 tilden st
4                149 e mosholu parkway room 414c
                         ...                    
482                      166 lockman ave room 16
483    657 castleton ave administrative building
484                  800 castleton ave 2nd floor
485                            669 castleton ave
486                            690 castleton ave
Name: site_address, Length: 487, dtype: object

In [9]:
fidelis.street = fidelis.street.str.lower()
fidelis.street = fidelis.street.str.replace("avenue", "ave")
fidelis.street = fidelis.street.str.replace("street", "st")
fidelis.street = fidelis.street.str.replace(".", "")
fidelis.street = fidelis.street.str.replace("-", "")
fidelis.street = fidelis.street.str.replace(" east ", " e ")
fidelis.street = fidelis.street.str.replace(" north ", " n ")
fidelis.street = fidelis.street.str.replace(" west ", " w ")
fidelis.street = fidelis.street.str.replace(" south ", " s ")
fidelis.street = fidelis.street.str.replace(" road", " rd")

fidelis.street

/var/folders/_4/_njb58q944xdbv6xcbzbzmqh0000gn/T/ipykernel_56730/2346575285.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fidelis.street = fidelis.street.str.replace(".", "")


0             65 broadway
1             50 broadway
2         2 washington st
3                253 s st
4          227 madison st
              ...        
193       392 seguine ave
194       80 n service rd
195    1000 n village ave
196          2 w park ave
197        250 fulton ave
Name: street, Length: 198, dtype: object

In [10]:
metroplus.address = metroplus.address.str.lower()
metroplus.address = metroplus.address.str.replace("avenue", "ave")
metroplus.address = metroplus.address.str.replace("street", "st")
metroplus.address = metroplus.address.str.replace(".", "")
metroplus.address = metroplus.address.str.replace("-", "")
metroplus.address = metroplus.address.str.replace(" east ", " e ")
metroplus.address = metroplus.address.str.replace(" north ", " n ")
metroplus.address = metroplus.address.str.replace(" west ", " w ")
metroplus.address = metroplus.address.str.replace(" south ", " s ")
metroplus.address = metroplus.address.str.replace(" road", " rd")

metroplus.address

/var/folders/_4/_njb58q944xdbv6xcbzbzmqh0000gn/T/ipykernel_56730/1703654167.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metroplus.address = metroplus.address.str.replace(".", "")


0                254 w 31st st, 9th floor new york 10001
1                        2015 madison ave new york 10035
2                            1363 jerome ave bronx 10457
3                          158 e 115th st new york 10029
4      1180 rev james a polite ave, room 230231b bron...
                             ...                        
947                        315 e 113th st new york 10029
948                           770 e 176th st bronx 10460
949              7520 astoria boulevard e elmhurst 11370
950                      1669 bedford ave brooklyn 11225
951                            40 elmont rd elmont 11003
Name: address, Length: 952, dtype: object

In [13]:
def clean_address(address):
    regex = re.findall("(\d+.+)",address)
    return regex[0]
    

In [14]:
healthfirst['clean_address'] = healthfirst.address.apply(clean_address)
healthfirst.clean_address = healthfirst.clean_address.str.lower()
healthfirst.clean_address = healthfirst.clean_address.str.replace("avenue", "ave")
healthfirst.clean_address = healthfirst.clean_address.str.replace("street", "st")
healthfirst.clean_address = healthfirst.clean_address.str.replace(".", "")
healthfirst.clean_address = healthfirst.clean_address.str.replace("-", "")
healthfirst.clean_address = healthfirst.clean_address.str.replace(",", "")
healthfirst.clean_address = healthfirst.clean_address.str.replace(" east ", " e ")
healthfirst.clean_address = healthfirst.clean_address.str.replace(" north ", " n ")
healthfirst.clean_address = healthfirst.clean_address.str.replace(" west ", " w ")
healthfirst.clean_address = healthfirst.clean_address.str.replace(" south ", " s ")
healthfirst.clean_address = healthfirst.clean_address.str.replace(" road", " rd")

healthfirst.clean_address = healthfirst.clean_address.str.split(' ').str[0] + " " + healthfirst.clean_address.str.split(' ').str[1] + " " + healthfirst.clean_address.str.split(' ').str[2]

/var/folders/_4/_njb58q944xdbv6xcbzbzmqh0000gn/T/ipykernel_56730/2116785916.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  healthfirst.clean_address = healthfirst.clean_address.str.replace(".", "")


In [15]:
healthfirst.clean_address

0             64 w 35th
1              7 w 30th
2            115 w 31st
3            122 w 27th
4           305 7th ave
             ...       
345       7a mansion st
346            230 n rd
347       20 crystal st
348    368 broadway 2nd
349    340/300 aaron ct
Name: clean_address, Length: 350, dtype: object

In [195]:
healthfirst.address

,name,field,phone,address,zip,city,clean_address
158,ASTOR SVCS FOR CHILDREN & FAMILIES,Mental Health Services,718-231-3400,"ASTOR AT 09X361 HG 200 W 167TH ST, BRONX, NY, ...",10452,BRONX,09x361 hg 200
167,ASTOR SERVICES FOR CHILD & FAM,Mental Health Services,845-452-6077,ASTOR HIGHBRIDGE CLINIC 1419 SHAKESPEARE AVEOF...,10452,BRONX,1419 shakespeare aveofc1


In [22]:
healthfirst.query('address.str.contains("inwood",case=False)')

,name,field,phone,address,zip,city,clean_address


In [19]:
nycClinics.query('agency_name == "Association to Benefit Children"')

,program_region,program_county,sponsor_name,agency_name,site_name,main_site,site_address,site_city,site_state,site_zip_code,site_county,population_simple,type,full_address,lat,long,geometry,clean_address
1,New York City,Bronx,Association to Benefit Children,Association to Benefit Children,Fast Break - The Bronx,Y,1366 inwood ave,Bronx,NY,10452,Bronx,Children & Adolescents,Community,"1366 Inwood Avenue, Bronx, NY 10452",40.839864,-73.918820,"[-73.9188205, 40.8398641]",1366 inwood ave
252,New York City,New York,Association to Benefit Children,Association to Benefit Children,ABC @ Joan of Arc Complex,N,154 w 93rd st room b21,New York,NY,10025,New York,Children & Adolescents,Community,"154 West 93rd Street Room B21, New York, NY 10025",40.791256,-73.971111,"[-73.9711111, 40.7912562]",154 w 93rd
253,New York City,New York,Association to Benefit Children,Association to Benefit Children,ABC Fast Break Yorkville,N,421 e 88th st room 402,New York,NY,10128,New York,Adolescents,Community,"421 East 88th Street Room 402, New York, NY 10128",40.777923,-73.947325,"[-73.9473251, 40.7779225]",421 e 88th
254,New York City,New York,Association to Benefit Children,Association to Benefit Children,Children's Mobile Mental Health Clinic,Y,1841 park ave,New York,NY,10035,New York,Children & Adolescents,Community,"1841 Park Avenue, New York, NY 10035",40.805679,-73.938049,"[-73.9380487, 40.8056792]",1841 park ave
255,New York City,New York,Association to Benefit Children,Association to Benefit Children,Fast Break Murry Bergtraum,N,411 pearl st rooms 137h & 229,New York,NY,10038,New York,Adolescents,Community,"411 Pearl Street Rooms 137H & 229, New York, N...",40.711241,-74.001442,"[-74.0014423, 40.7112411]",411 pearl st
256,New York City,New York,Association to Benefit Children,Association to Benefit Children,Fast Break at P.S. 206,N,508 e 120th st room 427,New York,NY,10035,New York,Children,School,"508 East 120th Street Room 427, New York, NY 1...",40.796906,-73.929978,"[-73.9299784, 40.7969061]",508 e 120th
257,New York City,New York,Association to Benefit Children,Association to Benefit Children,Fast Break 115,Y,158 e 115th st 1st floor,New York,NY,10029,New York,Children & Adolescents,Community,"158 East 115th Street 1st Floor, New York, NY ...",40.797615,-73.941907,"[-73.9419066, 40.797615]",158 e 115th


In [18]:
nycClinics.query('~clean_address.isin(@healthfirst.clean_address)')

,program_region,program_county,sponsor_name,agency_name,site_name,main_site,site_address,site_city,site_state,site_zip_code,site_county,population_simple,type,full_address,lat,long,geometry,clean_address
1,New York City,Bronx,Association to Benefit Children,Association to Benefit Children,Fast Break - The Bronx,Y,1366 inwood ave,Bronx,NY,10452,Bronx,Children & Adolescents,Community,"1366 Inwood Avenue, Bronx, NY 10452",40.839864,-73.918820,"[-73.9188205, 40.8398641]",1366 inwood ave
2,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor @ Mott Hall V,N,1551 e 172nd st room 216c,Bronx,NY,10472,Bronx,Children & Adolescents,Community,"1551 East 172nd Street Room 216C, Bronx, NY 10472",40.831887,-73.877282,"[-73.8772819, 40.8318874]",1551 e 172nd
3,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor Clinic Treatment Program - Tilden,Y,750 tilden st,Bronx,NY,10467,Bronx,Children & Adolescents,Community,"750 Tilden Street, Bronx, NY 10467",40.876675,-73.862719,"[-73.862719, 40.8766745]",750 tilden st
4,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,"Astor Services @ MS 80, The Isabel Rooney School",N,149 e mosholu parkway room 414c,Bronx,NY,10467,Bronx,Adolescents,School,"149 East Mosholu Parkway Room 414C, Bronx, NY ...",40.876802,-73.881370,"[-73.8813704, 40.8768024]",149 e mosholu
5,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor at Bronxwood,N,700 e 179th st room 223,Bronx,NY,10467,Bronx,Children & Adolescents,Community,"700 East 179th Street Room 223, Bronx, NY 10467",40.846289,-73.889363,"[-73.8893633, 40.8462893]",700 e 179th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Port Richmond Satellite,N,166 lockman ave room 16,Staten Island,NY,10303,Richmond,Children,Community,"166 Lockman Avenue Room 16, Staten Island, NY ...",40.633147,-74.162037,"[-74.16203709999999, 40.6331468]",166 lockman ave
483,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Pouch Satellite,N,657 castleton ave administrative building,Staten Island,NY,10301,Richmond,Children,Community,"657 Castleton Avenue Administrative Building, ...",40.635312,-74.103344,"[-74.1033443, 40.6353117]",657 castleton ave
484,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,SIMHS Chait Clinic of RUMC @ Pediat,N,800 castleton ave 2nd floor,Staten Island,NY,10310,Richmond,Children & Adolescents,School,"800 Castleton Avenue 2nd Floor, Staten Island,...",40.634491,-74.109268,"[-74.1092682, 40.63449079999999]",800 castleton ave
485,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Staten Island MH Society CHAIT Clinic of RUMC,Y,669 castleton ave,Staten Island,NY,10301,Richmond,Children & Adolescents,School,"669 Castleton Avenue, Staten Island, NY 10301",40.635264,-74.103762,"[-74.10376219999999, 40.6352635]",669 castleton ave


In [17]:
nycClinics['clean_address'] = nycClinics.site_address.str.split(' ').str[0] + " " + nycClinics.site_address.str.split(' ').str[1] + " " + nycClinics.site_address.str.split(' ').str[2]
fidelis['clean_address'] = fidelis.street.str.split(' ').str[0] + " " + fidelis.street.str.split(' ').str[1] + " " + fidelis.street.str.split(' ').str[2]
metroplus['clean_address'] = metroplus.address.str.split(' ').str[0] + " " + metroplus.address.str.split(' ').str[1] + " " + metroplus.address.str.split(' ').str[2]

In [139]:
display(nycClinics.clean_address.sample(5))
display(fidelis.clean_address.sample(5))
display(metroplus.clean_address.sample(5))

150    600 kingston ave
38       928 simpson st
19       730 bryant ave
310          351 w 18th
186       613 beach 9th
Name: clean_address, dtype: object

167        2058 jerome ave
95              312 e 94th
31              230 w 17th
143    915 westchester ave
164        1910 arthur ave
Name: clean_address, dtype: object

847         1622 central ave
319      one brooklyn health
43               499 w 133rd
825           239 n broadway
772    221 jericho turnpike,
Name: clean_address, dtype: object

In [141]:
fidelis.query('clean_address.isin(@nycClinics.clean_address)')

,name,street,cityzip,phone,city,zip,match,clean_address
0,The Epilepsy Institute dba Epilepsy Foundation...,65 broadway,"New York, NY, 10006",(212) 677-8550,New York,10006,None,NaN
1,The Mental Health Association of New York City...,50 broadway,"New York, NY, 10004",(212) 254-0333,New York,10004,None,NaN
3,"Hamilton Madison House, Inc.",253 s st,"New York, NY, 10002",(212) 720-4570,New York,10002,"(100, -, Hamilton-Madison House, Inc.)",253 s st
4,Gouverneur Diagnostic and Treatment Center,227 madison st,"New York, NY, 10002",(212) 238-7897,New York,10002,None,227 madison st
5,University Settlement Society Of New York,184 eldridge st,"New York, NY, 10002",(212) 453-4522,New York,10002,"(100, -, University Settlement Society of New ...",184 eldridge st
...,...,...,...,...,...,...,...,...
170,St Barnabas Hospital,260 e 188th st,"Bronx, NY, 10458",(718) 901-9109,Bronx,10458,"(100, -, St. Barnabas Hospital)",260 e 188th
177,Riverdale Mental Health Center,5676 riverdale ave,"Bronx, NY, 10471",(718) 796-5300,Bronx,10471,"(100, -, Riverdale Mental Health Center)",5676 riverdale ave
181,Long Island Jewish Medical Center,7559 263rd st,"Glen Oaks, NY, 11004",(718) 470-8100,Glen Oaks,11004,"(100, -, Long Island Jewish Medical Center)",7559 263rd st
182,Zucker Hillside Hospital,7559 263rd st,"Glen Oaks, NY, 11004",(718) 470-8100,Glen Oaks,11004,None,7559 263rd st


In [142]:
metroplus.query('clean_address.isin(@nycClinics.clean_address)')

,row,name,meta,address,phone,zip,match,clean_address
3,4,ASSOCIATION TO BENEFIT CHILDREN,Distance: 4.45 miles\n158 East 115th Street Ne...,158 e 115th st new york 10029,(646)459-6165,10029,"(23, -, Association to Benefit Children)",158 e 115th
4,5,ASTOR SERVICES FOR CHILDREN & FAMILIES,Distance: 7.56 miles\n1180 Rev James A Polite ...,"1180 rev james a polite ave, room 230231b bron...",(718)231-3400,10459,"(27, -, Astor Services for Children & Families)",1180 rev james
8,9,ASTOR SERVICES FOR CHILDREN & FAMILIES,"Distance: 8.81 miles\n577 East 179th Street, R...","577 e 179th st, rooms 122 and 201 bronx 10452",(718)231-3400,10452,"(27, -, Astor Services for Children & Families)",577 e 179th
13,14,BRONXCARE HEALTH SYSTEM,Distance: 7.49 miles\n1276 Fulton Ave Bronx 10...,1276 fulton ave bronx 10456,(718)992-7669,10456,"(19, -, BronxCare Health System)",1276 fulton ave
14,15,"CATHOLIC CHARITIES NEIGHBORHOOD SERVICES, INC.",Distance: 8.56 miles\n1623 Flatbush Ave Brookl...,1623 flatbush ave brooklyn 11210,(718)722-6258,11210,"(27, -, Catholic Charities Neighborhood Servic...",1623 flatbush ave
...,...,...,...,...,...,...,...,...
944,1026,UNION SETTLEMENT ASSOCIATION DBA JAMES WELDON ...,Distance: 4.48 miles\n210 East 115th Street Ap...,210 e 115th st apt 1a new york 10029,(212)828-6144,10029,"(22, -, Union Settlement Association, Inc.)",210 e 115th
946,1028,UNION SETTLEMENT/JOHNSON COUNSELING CENTER,Distance: 3.67 miles\n1775 Third Avenue New Yo...,1775 third ave new york 10029,(212)828-6144,10029,"(41, -, Union Settlement Association, Inc.)",1775 third ave
947,1029,UNION SETTLEMENT/JOHNSON COUNSELING CENTER,Distance: 4.43 miles\n315 East 113th Street Ne...,315 e 113th st new york 10029,(212)828-6144,10029,"(41, -, Union Settlement Association, Inc.)",315 e 113th
948,1030,"VIP COMMUNITY SERVICES, INC.",Distance: 8.55 miles\n770 E 176th St Bronx 104...,770 e 176th st bronx 10460,(718)542-8700,10460,"(32, -, Inwood Community Services, Inc.)",770 e 176th


In [68]:
nycClinics[nycClinics.agency_name.str.contains('greenwich',case=False)]

,program_region,program_county,sponsor_name,agency_name,site_name,main_site,site_address,site_city,site_state,site_zip_code,site_county,population_simple,type,full_address,lat,long,geometry,clean_address
271,New York City,New York,"Greenwich House, Inc.","Greenwich House, Inc.",The Center for Resiliency and Wellness,Y,27 barrow st 3rd floor,New York,NY,10014,New York,Children & Adolescents,Community,"27 Barrow Street 3rd Floor, New York, NY 10014",40.732273,-74.003066,"[-74.00306590000001, 40.7322731]",27 barrow st


In [212]:
def find_match(name):
    highest_ratio = 0
    for agency in nycClinics.agency_name:
        ratio = fuzz.token_set_ratio(name, agency)
        if ratio > highest_ratio:
            highest_ratio = ratio
            match = agency
    
    for site in nycClinics.site_name:
        ratio = fuzz.token_set_ratio(name, site)
        if ratio > highest_ratio:
            highest_ratio = ratio
            match = site
    
    if highest_ratio > 90:
        return match
    else:
        return None
    
def find_match_ratio(name):
    highest_ratio = 0
    for agency in nycClinics.agency_name:
        ratio = fuzz.token_set_ratio(name, agency)
        if ratio > highest_ratio:
            highest_ratio = ratio
            match = agency
    
    for site in nycClinics.site_name:
        ratio = fuzz.token_set_ratio(name, site)
        if ratio > highest_ratio:
            highest_ratio = ratio
            match = site
    
    if highest_ratio > 90:
        return highest_ratio
    else:
        return None

In [130]:
fidelis['match'] = fidelis.name.apply(find_match)
# metroplus['match'] = metroplus.name.apply(find_match)
# healthfirst['match'] = healthfirst.name.apply(find_match)

In [131]:
fidelis.head()

,name,street,cityzip,phone,city,zip,match
0,The Epilepsy Institute dba Epilepsy Foundation...,65 Broadway,"New York, NY, 10006",(212) 677-8550,New York,10006,None
1,The Mental Health Association of New York City...,50 Broadway,"New York, NY, 10004",(212) 254-0333,New York,10004,None
2,Community Access Inc - Bronx Shelter,2 Washington St,"New York, NY, 10004",(212) 780-1400,New York,10004,None
3,"Hamilton Madison House, Inc.",253 South St,"New York, NY, 10002",(212) 720-4570,New York,10002,"(100, -, Hamilton-Madison House, Inc.)"
4,Gouverneur Diagnostic and Treatment Center,227 Madison St,"New York, NY, 10002",(212) 238-7897,New York,10002,None


In [132]:
fidelis[['name','match']].sample(10)

,name,match
117,Palladia Inc dba Comprehensive Treatment Insti...,None
16,Jewish Board of Family and Children's Services...,"(100, -, Jewish Board of Family & Children's S..."
147,BronxCare Health System,"(100, -, BronxCare Health System)"
76,NYU Langone Hospital - Brooklyn,None
39,Bridging Access to Care Inc,"(100, -, Bridging Access to Care, Inc.)"
126,"New Horizon Counseling Center, Inc.","(100, -, New Horizon Counseling Center, Inc., ..."
181,Long Island Jewish Medical Center,"(100, -, Long Island Jewish Medical Center)"
157,Flushing Hospital Medical Center,"(100, -, New York Flushing Hospital and Medica..."
145,WellLife Network Inc.,None
142,Community Association of Progressive Dominican...,None


In [119]:
nycClinics[nycClinics.site_name.str.contains('zucker',case=False,na=False)]

,program_region,program_county,sponsor_name,agency_name,site_name,main_site,site_address,site_city,site_state,site_zip_code,site_county,population_simple,type,full_address,lat,long,geometry
441,New York City,Queens,"Northwell Healthcare, Inc./Northwell Quality a...",Long Island Jewish Medical Center,The Zucker Hillside Ambulatory Care Clinic,Y,"75-59 263rd Street Amb Care Pavilion Bsmt, 1st...",Glen Oaks,NY,11004,Queens,Children & Adolescents,Community,"75-59 263rd Street Amb Care Pavilion Bsmt, 1st...",40.752736,-73.711487,"[-73.7114873, 40.7527356]"


In [228]:
def fidelis_address_match(clean_address):
    for address in fidelis.clean_address:
        if address == clean_address:
            return 1
        
def metroplus_address_match(clean_address):
    for address in metroplus.clean_address:
        if address == clean_address:
            return 1
        
def healthfirst_address_match(clean_address):
    for address in healthfirst.clean_address:
        if address == clean_address:
            return 1

In [229]:
nycClinics['fidelis_address_match'] = nycClinics.clean_address.apply(fidelis_address_match)
nycClinics['metroplus_address_match'] = nycClinics.clean_address.apply(metroplus_address_match)
nycClinics['healthfirst_address_match'] = nycClinics.clean_address.apply(healthfirst_address_match)
nycClinics.head()

,program_region,program_county,sponsor_name,agency_name,site_name,main_site,site_address,site_city,site_state,site_zip_code,...,population_simple,type,full_address,lat,long,geometry,clean_address,fidelis_address_match,metroplus_address_match,healthfirst_address_match
0,New York City,Bronx,"Acacia Network, Inc.","PROMESA, Inc.",Westchester Center of Excellence,Y,915 westchester ave 2nd floor,Bronx,NY,10459,...,Children & Adolescents,Community,"915 Westchester Avenue 2nd Floor, Bronx, NY 10459",40.821566,-73.898408,"[-73.8984076, 40.8215662]",915 westchester ave,1.0,1.0,1.0
1,New York City,Bronx,Association to Benefit Children,Association to Benefit Children,Fast Break - The Bronx,Y,1366 inwood ave,Bronx,NY,10452,...,Children & Adolescents,Community,"1366 Inwood Avenue, Bronx, NY 10452",40.839864,-73.918820,"[-73.9188205, 40.8398641]",1366 inwood ave,1.0,1.0,NaN
2,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor @ Mott Hall V,N,1551 e 172nd st room 216c,Bronx,NY,10472,...,Children & Adolescents,Community,"1551 East 172nd Street Room 216C, Bronx, NY 10472",40.831887,-73.877282,"[-73.8772819, 40.8318874]",1551 e 172nd,NaN,1.0,NaN
3,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor Clinic Treatment Program - Tilden,Y,750 tilden st,Bronx,NY,10467,...,Children & Adolescents,Community,"750 Tilden Street, Bronx, NY 10467",40.876675,-73.862719,"[-73.862719, 40.8766745]",750 tilden st,NaN,1.0,NaN
4,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,"Astor Services @ MS 80, The Isabel Rooney School",N,149 e mosholu parkway room 414c,Bronx,NY,10467,...,Adolescents,School,"149 East Mosholu Parkway Room 414C, Bronx, NY ...",40.876802,-73.881370,"[-73.8813704, 40.8768024]",149 e mosholu,NaN,1.0,NaN


In [230]:
def fidelis_name_match(agency_name):
    highest_ratio = 0
    for name in fidelis.name:
        ratio = fuzz.token_set_ratio(agency_name, name)
        if ratio > highest_ratio:
            highest_ratio = ratio
            match = name
    
    if highest_ratio > 90:
        return match
    else:
        return None

def healthfirst_name_match(agency_name):
    highest_ratio = 0
    for name in healthfirst.name:
        ratio = fuzz.token_set_ratio(agency_name, name)
        if ratio > highest_ratio:
            highest_ratio = ratio
            match = name
    
    if highest_ratio > 90:
        return match
    else:
        return None
    
def metroplus_name_match(agency_name):
    highest_ratio = 0
    for name in metroplus.name:
        ratio = fuzz.token_set_ratio(agency_name, name)
        if ratio > highest_ratio:
            highest_ratio = ratio
            match = name
    
    if highest_ratio > 90:
        return match
    else:
        return None

In [231]:
nycClinics['fidelis_name_match'] = nycClinics.agency_name.apply(fidelis_name_match)
nycClinics['metroplus_name_match'] = nycClinics.agency_name.apply(metroplus_name_match)
nycClinics['healthfirst_name_match'] = nycClinics.agency_name.apply(healthfirst_name_match)

In [232]:
nycClinics

,program_region,program_county,sponsor_name,agency_name,site_name,main_site,site_address,site_city,site_state,site_zip_code,...,lat,long,geometry,clean_address,fidelis_address_match,metroplus_address_match,healthfirst_address_match,fidelis_name_match,metroplus_name_match,healthfirst_name_match
0,New York City,Bronx,"Acacia Network, Inc.","PROMESA, Inc.",Westchester Center of Excellence,Y,915 westchester ave 2nd floor,Bronx,NY,10459,...,40.821566,-73.898408,"[-73.8984076, 40.8215662]",915 westchester ave,1.0,1.0,1.0,"Promesa, Inc.","PROMESA, INC.",PROMESA INC
1,New York City,Bronx,Association to Benefit Children,Association to Benefit Children,Fast Break - The Bronx,Y,1366 inwood ave,Bronx,NY,10452,...,40.839864,-73.918820,"[-73.9188205, 40.8398641]",1366 inwood ave,1.0,1.0,NaN,Association to Benefit Children,ASSOCIATION TO BENEFIT CHILDREN,ASSOCIATION TO BENEFIT CHLDRN
2,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor @ Mott Hall V,N,1551 e 172nd st room 216c,Bronx,NY,10472,...,40.831887,-73.877282,"[-73.8772819, 40.8318874]",1551 e 172nd,NaN,1.0,NaN,Astor Services For Children & Families,ASTOR SERVICES FOR CHILDREN & FAMILIES,ASTOR SVCS FOR CHILDREN & FAMILIES
3,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,Astor Clinic Treatment Program - Tilden,Y,750 tilden st,Bronx,NY,10467,...,40.876675,-73.862719,"[-73.862719, 40.8766745]",750 tilden st,NaN,1.0,NaN,Astor Services For Children & Families,ASTOR SERVICES FOR CHILDREN & FAMILIES,ASTOR SVCS FOR CHILDREN & FAMILIES
4,New York City,Bronx,Astor Services for Children & Families,Astor Services for Children & Families,"Astor Services @ MS 80, The Isabel Rooney School",N,149 e mosholu parkway room 414c,Bronx,NY,10467,...,40.876802,-73.881370,"[-73.8813704, 40.8768024]",149 e mosholu,NaN,1.0,NaN,Astor Services For Children & Families,ASTOR SERVICES FOR CHILDREN & FAMILIES,ASTOR SVCS FOR CHILDREN & FAMILIES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Port Richmond Satellite,N,166 lockman ave room 16,Staten Island,NY,10303,...,40.633147,-74.162037,"[-74.16203709999999, 40.6331468]",166 lockman ave,NaN,NaN,NaN,None,None,None
483,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Pouch Satellite,N,657 castleton ave administrative building,Staten Island,NY,10301,...,40.635312,-74.103344,"[-74.1033443, 40.6353117]",657 castleton ave,NaN,NaN,NaN,None,None,None
484,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,SIMHS Chait Clinic of RUMC @ Pediat,N,800 castleton ave 2nd floor,Staten Island,NY,10310,...,40.634491,-74.109268,"[-74.1092682, 40.63449079999999]",800 castleton ave,NaN,NaN,NaN,None,None,None
485,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Staten Island MH Society CHAIT Clinic of RUMC,Y,669 castleton ave,Staten Island,NY,10301,...,40.635264,-74.103762,"[-74.10376219999999, 40.6352635]",669 castleton ave,NaN,1.0,NaN,None,None,None


In [241]:
nycClinics.query('type == "Community" and healthfirst_address_match.isnull() and healthfirst_name_match.isnull()')

,program_region,program_county,sponsor_name,agency_name,site_name,main_site,site_address,site_city,site_state,site_zip_code,...,lat,long,geometry,clean_address,fidelis_address_match,metroplus_address_match,healthfirst_address_match,fidelis_name_match,metroplus_name_match,healthfirst_name_match
33,New York City,Bronx,BronxCare Health System,BronxCare Health System,BronxCare Child Study Center,Y,1276 fulton ave,Bronx,NY,10456,...,40.831581,-73.902362,"[-73.9023625, 40.8315806]",1276 fulton ave,1.0,1.0,NaN,BronxCare Health System,BRONXCARE HEALTH SYSTEM,None
34,New York City,Bronx,Cardinal McCloskey School & Home for Children ...,C McCloskey SHFC dba C McCloskey Comm Svcs,Hayden House,N,155 n highland ave,Ossining,NY,10562,...,41.168608,-73.863913,"[-73.8639135, 41.1686077]",155 n highland,NaN,NaN,NaN,None,None,None
35,New York City,Bronx,Cardinal McCloskey School & Home for Children ...,C McCloskey SHFC dba C McCloskey Comm Svcs,McCloskey Cares,Y,529 cortlandt ave 5th floor,Bronx,NY,10451,...,40.816126,-73.919991,"[-73.91999129999999, 40.816126]",529 cortlandt ave,NaN,NaN,NaN,None,None,None
37,New York City,Bronx,Gotham Health FQHC,NYC-HHC Morrisania Diagnostic & Treatment Ctr,Morrisania Diagnostic and Treatment Center,Y,1225 gerard ave,Bronx,NY,10452,...,40.836287,-73.920229,"[-73.9202294, 40.8362867]",1225 gerard ave,1.0,1.0,NaN,None,None,None
39,New York City,Bronx,Jewish Board of Family and Children's Services...,Jewish Board of Family & Children's Services,Grand Concourse Counseling Center,Y,2488 grand concourse 2nd floor,Bronx,NY,10458,...,40.862105,-73.896787,"[-73.896787, 40.862105]",2488 grand concourse,NaN,1.0,NaN,Jewish Board of Family and Children's Services...,JEWISH BOARD OF FAMILY AND CHILDREN'S SERVICES...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Kingsley Satellite,N,10 kingsley place room 207,Staten Island,NY,10301,...,40.635931,-74.088992,"[-74.0889918, 40.6359312]",10 kingsley place,NaN,NaN,NaN,None,None,None
478,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Osgood Satellite,N,16 osgood ave room 7,Staten Island,NY,10304,...,40.618748,-74.085366,"[-74.0853664, 40.6187476]",16 osgood ave,NaN,NaN,NaN,None,None,None
482,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Port Richmond Satellite,N,166 lockman ave room 16,Staten Island,NY,10303,...,40.633147,-74.162037,"[-74.16203709999999, 40.6331468]",166 lockman ave,NaN,NaN,NaN,None,None,None
483,New York City,Richmond,Richmond Medical Center dba Richmond Universit...,Richmond Medical Center dba Richmond UMC,Pouch Satellite,N,657 castleton ave administrative building,Staten Island,NY,10301,...,40.635312,-74.103344,"[-74.1033443, 40.6353117]",657 castleton ave,NaN,NaN,NaN,None,None,None


In [199]:
metroplus['address_match'] = metroplus.clean_address.apply(address_match)
fidelis['address_match'] = fidelis.clean_address.apply(address_match)
healthfirst['address_match'] = fidelis.clean_address.apply(address_match)

In [213]:
fidelis['name_match'] = fidelis.name.apply(find_match)
fidelis['name_match_ratio'] = fidelis.name.apply(find_match_ratio)

In [221]:
healthfirst['name_match'] = healthfirst.name.apply(find_match)
healthfirst['name_match_ratio'] = healthfirst.name.apply(find_match_ratio)

In [214]:
fidelis

,name,street,cityzip,phone,city,zip,match,clean_address,address_match,name_match,name_match_ratio
0,The Epilepsy Institute dba Epilepsy Foundation...,65 broadway,"New York, NY, 10006",(212) 677-8550,New York,10006,None,NaN,NaN,None,NaN
1,The Mental Health Association of New York City...,50 broadway,"New York, NY, 10004",(212) 254-0333,New York,10004,None,NaN,NaN,None,NaN
2,Community Access Inc - Bronx Shelter,2 washington st,"New York, NY, 10004",(212) 780-1400,New York,10004,None,2 washington st,NaN,None,NaN
3,"Hamilton Madison House, Inc.",253 s st,"New York, NY, 10002",(212) 720-4570,New York,10002,"(100, -, Hamilton-Madison House, Inc.)",253 s st,1.0,"Hamilton-Madison House, Inc.",100.0
4,Gouverneur Diagnostic and Treatment Center,227 madison st,"New York, NY, 10002",(212) 238-7897,New York,10002,None,227 madison st,1.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
193,Staten Island University Hospital,392 seguine ave,"Staten Island, NY, 10309",(718) 226-2753,Staten Island,10309,"(100, -, Staten Island University Hospital)",392 seguine ave,NaN,Staten Island University Hospital,100.0
194,North Shore Child & Family Guidance Center,80 n service rd,"Manhasset, NY, 11030",(518) 626-1971,Manhasset,11030,None,80 n service,NaN,None,NaN
195,Mercy Medical Center,1000 n village ave,"Rockville Centre, NY, 11570",(516) 705-2525,Rockville Centre,11570,None,1000 n village,NaN,None,NaN
196,Long Beach Reach Inc,2 w park ave,"Long Beach, NY, 11561",(516) 889-2332,Long Beach,11561,None,2 w park,NaN,None,NaN
